In [1]:
import nbformat as nbf
from pathlib import Path

nb = nbf.v4.new_notebook()
nb["metadata"]["colab"] = {"provenance": []}
cells=[]

cells += [
nbf.v4.new_markdown_cell("# 07 — Explainability: SHAP (XGB) + Linear Coeffs"),
nbf.v4.new_code_cell("%pip -q install shap xgboost scikit-learn pandas numpy matplotlib"),
nbf.v4.new_code_cell(r'''
import shap, numpy as np, pandas as pd, matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# โหลดชุดฟีเจอร์ (ถ้ามี sentiment จะใช้ไฟล์นั้นก่อน)
for f in ["dataset_features_labels_with_sentiment.csv","dataset_features_labels.csv"]:
    try:
        df = pd.read_csv(f, index_col=0, parse_dates=True); print("Loaded:", f); break
    except: pass
assert 'df' in globals(), "ไม่พบ dataset_features_labels*.csv — รัน 02/06 ก่อน"

FEATURES = [c for c in df.columns if c!="Target"]
X, y = df[FEATURES].values, df["Target"].values
n=len(df); n_train=int(n*0.7); n_val=int(n*0.85)
X_train, y_train = X[:n_val], y[:n_val]
X_test,  y_test  = X[n_val:], y[n_val:]

# --- XGB ---
xgb = XGBRegressor(
    n_estimators=400, max_depth=4, learning_rate=0.05,
    subsample=0.9, colsample_bytree=0.9, random_state=42
)
xgb.fit(X_train, y_train)
print("XGB R2:", r2_score(y_test, xgb.predict(X_test)))

explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(X_test)

plt.figure(figsize=(9,5))
shap.summary_plot(shap_values, features=X_test, feature_names=FEATURES, show=False)
plt.tight_layout(); plt.savefig("shap_summary_xgb.png", dpi=150); plt.show()
print("Saved shap_summary_xgb.png")

# --- Linear ---
lin = LinearRegression().fit(X_train, y_train)
print("Linear R2:", r2_score(y_test, lin.predict(X_test)))
coef_df = pd.DataFrame({"feature": FEATURES, "coef": lin.coef_}).sort_values("coef", ascending=False)
coef_df.to_csv("linear_coefficients.csv", index=False)
coef_df.head(10)
''')
]

nb["cells"]=cells
Path("07_explainability_shap.ipynb").write_text(nbf.writes(nb), encoding="utf-8")
print("✅ Created 07_explainability_shap.ipynb")


✅ Created 07_explainability_shap.ipynb
